## Useful links
https://medium.com/rapids-ai/bursting-data-science-workloads-to-gpus-on-google-cloud-platform-with-dask-cloud-provider-685be1eff204

https://cloudprovider.dask.org/en/latest/gcp.html

https://cloudprovider.dask.org/en/latest/packer.html

In [ ]:
import os
import json

from dask_cloudprovider.gcp import GCPCluster
from dask.distributed import Client
import dask.config

## Create cluster and wait

In [ ]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../../oxeo-main-prefect.json"

dask.config.set(
    **{
        "cloudprovider.gcp.projectid": "oxeo-main",
        "public_ingress": True,
    }
)

In [ ]:
with GCPCluster(
    zone="europe-west4-a",
    machine_type="n1-highmem-2",
    n_workers=1,
    docker_image="eu.gcr.io/oxeo-main/oxeo-flows-dask:latest",
) as cluster:
    print(cluster)
    input("waiting")
    print("done")

## Create packer config

In [ ]:
# Init
cloud_init = GCPCluster.get_cloud_init(
    zone="europe-west4-a",
    machine_type="n1-highmem-2",
    n_workers=1,
    docker_image="eu.gcr.io/oxeo-main/oxeo-flows-dask:latest",
)
# print(cloud_init_config)

In [ ]:
cloud_init = """
#cloud-config
# Bootstrap
packages:
  - apt-transport-https
  - ca-certificates
  - curl
  - gnupg-agent
  - software-properties-common
  - ubuntu-drivers-common

# Enable ipv4 forwarding, required on CIS hardened machines
write_files:
  - path: /etc/sysctl.d/enabled_ipv4_forwarding.conf
    content: |
      net.ipv4.conf.all.forwarding=1

# create the docker group
groups:
  - docker

# Add default auto created user to docker group
system_info:
  default_user:
    groups: [docker]

runcmd:
  # Install Docker
  - curl -fsSL https://download.docker.com/linux/ubuntu/gpg | apt-key add -
  - add-apt-repository "deb [arch=amd64] https://download.docker.com/linux/ubuntu $(lsb_release -cs) stable"
  - apt-get update -y
  - apt-get install -y docker-ce docker-ce-cli containerd.io
  - systemctl start docker
  - systemctl enable docker
  - gcloud auth configure-docker
  
  # Run container
  - 'docker run --net=host   eu.gcr.io/oxeo-main/oxeo-flows-dask:latest dask-scheduler --version'
"""

In [ ]:
packer_config = {
    "builders": [
        {
            "type": "googlecompute",
            "project_id": "oxeo-main",
            "source_image": "ubuntu-2004-focal-v20211102",
            "ssh_username": "packer",
            "zone": "europe-west4-a",
            "on_host_maintenance": "TERMINATE",
            "disk_size": "30",
            "machine_type": "n1-highmem-2",
            "metadata": {"user-data": cloud_init},
        }
    ],
    "provisioners": [
        {
            "type": "shell",
            "inline": [
                "echo 'Waiting for cloud-init'; while [ ! -f /var/lib/cloud/instance/boot-finished ]; do sleep 1; done; echo 'Done'"
            ],
        }
    ],
}

In [ ]:
with open("../packer.json", "w") as fh:
    fh.write(json.dumps(packer_config))